In [1]:
import numpy as np
import pandas as pd

In [2]:
xl = pd.ExcelFile('wartosci_walorow_12_22.xlsx')
sheets = xl.sheet_names

merged = xl.parse(sheets.pop(0))
print(sheets)

for sheet in sheets:
    data = xl.parse(sheet)    
    merged = merged.merge(data, on='Date', how='left') 

xl.close()

assets = merged.columns[1:]
price_changes = merged[assets].pct_change()
price_changes.set_index(merged['Date'], inplace=True)

euro_assets = ['ACWI', 'GovBondG7', 'GlobalCorporate', 'Gold', 'GABHDG', 'EURO_STOXX_Leverage', 'Fallen_Angels']
pln_assets = ['Mwig40tr', 'TBSP', 'EDO']

pln = merged[euro_assets].apply(lambda x: x * merged['EURPLN'])
pln_chng = pln.pct_change()
pln_chng.set_index(merged['Date'], inplace=True)
pln_chng = pd.merge(price_changes[pln_assets], pln_chng, left_index=True, right_index=True)

assets = pln_chng
assets.info()

['MSCI ACWI', 'MWITR', 'TBSP', 'EURPLN', 'EDO', 'Bloomberg Barclays Global Aggre', 'Gold spot price', 'Global aggregate euro hdg', 'EURO STOXX 50 Daily Leverage', 'Bloomberg Barclays Global Corpo']
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 456 entries, 1985-01-01 to 2022-12-01
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Mwig40tr             155 non-null    float64
 1   TBSP                 191 non-null    float64
 2   EDO                  225 non-null    float64
 3   ACWI                 254 non-null    float64
 4   GovBondG7            254 non-null    float64
 5   GlobalCorporate      123 non-null    float64
 6   Gold                 254 non-null    float64
 7   GABHDG               61 non-null     float64
 8   EURO_STOXX_Leverage  252 non-null    float64
 9   Fallen_Angels        78 non-null     float64
dtypes: float64(10)
memory usage: 55.4 KB


In [3]:
def shrinkedCovariance(returns: pd.DataFrame, w: int):
    std = np.diag(returns.std())*np.sqrt(12)
    corr = returns.corr('pearson')
    shrinked_corr = (1-w)*corr + np.ones_like(corr)*w 
    result = std @ shrinked_corr @ std
    return std,corr, shrinked_corr, result

def pstd(cov_matrix, weights):
    return np.sqrt(weights.T @ cov_matrix @ weights)

In [5]:
assets_names = ['TBSP', 'ACWI']

portfolio_returns = assets.iloc[-120:][assets_names]

std, corr, sh_corr, res = shrinkedCovariance(portfolio_returns,0.5)
cov = portfolio_returns.cov()*12

print (portfolio_returns.cov()*12)
print(std)
print(corr)
print(sh_corr)
print(res)
print('---')
print(pstd(cov, np.array([0.4, 0.6])))
print('---')
print(pstd(res, np.array([0.4, 0.6])))




          TBSP      ACWI
TBSP  0.002233 -0.000662
ACWI -0.000662  0.023129
[[0.04725282 0.        ]
 [0.         0.15208061]]
          TBSP      ACWI
TBSP  1.000000 -0.092152
ACWI -0.092152  1.000000
          TBSP      ACWI
TBSP  1.000000  0.453924
ACWI  0.453924  1.000000
          0         1
0  0.002233  0.003262
1  0.003262  0.023129
---
0.09146392619858458
---
0.10123873094081719


In [12]:
ret = assets.iloc[-60:][assets_names]
ret_24 = assets.iloc[-84:-24][assets_names]
ret_48 = assets.iloc[-108:-48][assets_names]
ret_150 = assets.iloc[-150:][assets_names]
print(ret.std()*np.sqrt(12))
print(ret_24.std()*np.sqrt(12))
print(ret_48.std()*np.sqrt(12))
print(ret_150.std()*np.sqrt(12))
print('---')

std, corr, sh_corr, res = shrinkedCovariance(ret,0)
std, corr, sh_corr, res_24 = shrinkedCovariance(ret_24,0)
std, corr, sh_corr, res_48 = shrinkedCovariance(ret_48,0)
std, corr, sh_corr, res_150 = shrinkedCovariance(ret_150,0)


print(pstd(res, np.array([0.4, 0.6])))
print(pstd(res_24, np.array([0.4, 0.6])))
print(pstd(res_48, np.array([0.4, 0.6])))
print(pstd(res_150, np.array([0.4, 0.6])))


print('---')
std, corr, sh_corr, res = shrinkedCovariance(ret,0.75)
std, corr, sh_corr, res_24 = shrinkedCovariance(ret_24,0.75)
std, corr, sh_corr, res_48 = shrinkedCovariance(ret_48,0.75)
std, corr, sh_corr, res_150 = shrinkedCovariance(ret_150,0.75)

print(pstd(res, np.array([0.4, 0.6])))
print(pstd(res_24, np.array([0.4, 0.6])))
print(pstd(res_48, np.array([0.4, 0.6])))
print(pstd(res_150, np.array([0.4, 0.6])))

TBSP    0.058287
ACWI    0.180158
dtype: float64
TBSP    0.019728
ACWI    0.156034
dtype: float64
TBSP    0.024732
ACWI    0.129339
dtype: float64
TBSP    0.044049
ACWI    0.150289
dtype: float64
---
0.10942527346768507
0.09450573239453121
0.07687067421906144
0.08952406152781506
---
0.126273078746576
0.09980622208789802
0.08496410489639229
0.10352840991202439
